# Full Analysis of Philippine Food prices from 2007 - 2025

## Aims:
    ### 1. Compare average prices across different regions for the same food item and year.
    ### 2. Examine how prices of a specific food item have changed over time within a region.
    ### 3. Identify regional price variations for different food items.
    ### 4. Predict future food prices based on historical data.

In [24]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro
from scipy.stats import kruskal
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import levene
from ipywidgets import interact
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import scikit_posthocs as sp
import ipywidgets as widgets
from IPython.display import display

In [25]:
#open the file that was already cleaned, analyzed, and saved.
file_path = "../Data/Processed/eda_results.csv"
df = pd.read_csv(file_path)

#### Inspect the data

In [26]:
# View the first few rows of the dataset
print(df.head())

                                 Region Province Food_Items  year  mean  \
0  Autonomous region in Muslim Mindanao  Basilan      beans  2007   NaN   
1  Autonomous region in Muslim Mindanao  Basilan      beans  2008   NaN   
2  Autonomous region in Muslim Mindanao  Basilan      beans  2009   NaN   
3  Autonomous region in Muslim Mindanao  Basilan      beans  2010   NaN   
4  Autonomous region in Muslim Mindanao  Basilan      beans  2011   NaN   

   median  Mode  Variance  Standard Deviation  IQR  
0     NaN   NaN       NaN                 NaN  NaN  
1     NaN   NaN       NaN                 NaN  NaN  
2     NaN   NaN       NaN                 NaN  NaN  
3     NaN   NaN       NaN                 NaN  NaN  
4     NaN   NaN       NaN                 NaN  NaN  


In [27]:
# View the columns and data types of the dataset
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91200 entries, 0 to 91199
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Region              91200 non-null  object 
 1   Province            91200 non-null  object 
 2   Food_Items          91200 non-null  object 
 3   year                91200 non-null  int64  
 4   mean                78875 non-null  float64
 5   median              78875 non-null  float64
 6   Mode                36432 non-null  float64
 7   Variance            76125 non-null  float64
 8   Standard Deviation  76125 non-null  float64
 9   IQR                 78875 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 7.0+ MB
None


In [28]:
# View the summary statistics of the dataset though this may not make sense as they are already described in the EDA report
print(df.describe())

               year          mean        median          Mode      Variance  \
count  91200.000000  78875.000000  78875.000000  36432.000000  76125.000000   
mean    2016.000000    105.451058    104.863522     53.431795    122.240405   
std        5.477256     78.042087     77.886750     36.845196    563.636799   
min     2007.000000      3.913333      3.910000      4.000000      0.000000   
25%     2011.000000     45.180000     44.880000     21.550000      2.445597   
50%     2016.000000     83.240417     82.675000     50.880000     11.208699   
75%     2021.000000    157.035417    156.402500     82.030000     60.107766   
max     2025.000000    472.291667    470.625000    220.310000  24723.623884   

       Standard Deviation           IQR  
count        76125.000000  78875.000000  
mean             6.382062      7.540924  
std              9.028331     10.880566  
min              0.000000      0.000000  
25%              1.563840      1.665000  
50%              3.347940      3.950

In [29]:
#check the shape of the dataset
print(df.shape)

(91200, 10)


In [30]:
# get the first and last readings for each food item
print(df['year'].agg(['min', 'max']))

min    2007
max    2025
Name: year, dtype: int64


For comparing average prices across regions for the same food item and year, the closing price is generally the best price to use. Here’s why:

1. Consistency: The closing price is considered the most representative of the market's consensus for that period. It accounts for the entire trading session and reflects both supply and demand dynamics over time.
2. Standard Usage: The closing price is the most widely used price in financial markets, meaning data and analysis are typically focused around this price.
3. Simplicity: It eliminates the noise created by intra-day fluctuations, focusing on the price at the end of the trading session, which is more relevant for long-term comparisons.

In [31]:
# Filter the dataframe to keep only rows where 'food_item' starts with 'c_'
df_filtered = df[df['Food_Items'].str.startswith('c_')]

print(df_filtered)

                                     Region         Province  Food_Items  \
19     Autonomous region in Muslim Mindanao          Basilan     c_beans   
20     Autonomous region in Muslim Mindanao          Basilan     c_beans   
21     Autonomous region in Muslim Mindanao          Basilan     c_beans   
22     Autonomous region in Muslim Mindanao          Basilan     c_beans   
23     Autonomous region in Muslim Mindanao          Basilan     c_beans   
...                                     ...              ...         ...   
90302                           Region XIII  Surigao del Sur  c_tomatoes   
90303                           Region XIII  Surigao del Sur  c_tomatoes   
90304                           Region XIII  Surigao del Sur  c_tomatoes   
90305                           Region XIII  Surigao del Sur  c_tomatoes   
90306                           Region XIII  Surigao del Sur  c_tomatoes   

       year       mean  median  Mode   Variance  Standard Deviation     IQR  
19     20

In [32]:
# Due to having 18 regions in the Philippines, instead of creating a graph for each region, we will create a graph for each food item
# and make an interactive plot with a dropdown to select the food item.

# Initialize the Dash app
app = dash.Dash(__name__)

# Dropdown options for food items, removing c_ prefix
food_item_options = [{'label': item.replace('c_', ''), 'value': item} for item in df_filtered['Food_Items'].unique()]

# Dropdown options for mean/median
mean_median_options = [
    {'label': 'mean', 'value': 'mean'},
    {'label': 'median', 'value': 'median'}
]

# App layout
app.layout = html.Div([
    html.H1("Region-wise Food Data Visualization"),
    
    # Dropdown for selecting food item
    dcc.Dropdown(
        id='food_item_dropdown',
        options=food_item_options,
        value='Apples',  # Default food item
        style={'width': '50%'}
    ),
    
    # Dropdown for selecting mean or median
    dcc.Dropdown(
        id='mean_median_dropdown',
        options=mean_median_options,
        value='mean',  # Default selection is 'mean'
        style={'width': '50%'}
    ),
    
    # Graph to display the data
    dcc.Graph(id='region_graph')
])

# Callback to update the graph based on selected food item and mean/median
@app.callback(
    dash.dependencies.Output('region_graph', 'figure'),
    [
        dash.dependencies.Input('food_item_dropdown', 'value'),
        dash.dependencies.Input('mean_median_dropdown', 'value')
    ]
)
def update_graph(selected_food_item, selected_stat):
    # Handle the case where selected_food_item is None
    if not selected_food_item:
        return px.line(title="No food item selected")
    
    # Filter the data based on the selected food item
    filtered_df = df_filtered[df_filtered['Food_Items'] == selected_food_item]
    
    # Remove the 'c_' prefix for display purposes in the title
    display_food_item = selected_food_item.replace('c_', '')
    
    # Find the first and last year in the filtered dataset
    first_year = filtered_df['year'].min()
    last_year = filtered_df['year'].max()

    # Create a plot based on the selected statistic (mean or median)
    fig = px.line(
        filtered_df, 
        x='year', 
        y=selected_stat, 
        color='Region', 
        title=f'{selected_stat.capitalize()} for {display_food_item} by Region and Year',
        labels={selected_stat: selected_stat.capitalize(), 'year': 'Year', 'Region': 'Region'}
    )

    # Extend the x-axis to include the last year
    fig.update_xaxes(
        tickmode='linear',  # Ensure all years are shown on the x-axis
        tick0=first_year,   # Start from the first year
        dtick=1,            # Increment by 1 year
        range=[first_year, last_year + 1]  # Extend the range to include the last year
    )
    
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8050, mode='inline', name="app")
# Run the Dash apps with unique URLs


The graph shows the mean (average) price or median price of different food items over time for different regions in the Philippines.
#### Key Observation
1. Trend - Generally, the price of all food items has been increasing over the years across most regions. There are some fluctuations and periods of stability for some items, but the overall trend is upward.
2. Regional Differences - There is significant variation in price of different food items across regions. Some regions consistently have higher prices than others. Price fluctuations also varies between regions. Some regions experience more volatile prices than others.

Factors like weather patterns, government policies, agriculture changes, food transportation and global market trends can influence these prices.

To know which regions has the highest and lowest price every year for every food item, max, min will be calculated. Then range is next to know how wide the difference between the highest and lowest price.

In [51]:
file_path = "../Data/Interim/cleaned_food_prices.csv"
df_range = pd.read_csv(file_path)

#dropping columns related to food price index
df_nofpi_range = df_range.drop(columns=['o_food_price_index', 'h_food_price_index', 'l_food_price_index', 'c_food_price_index', 'inflation_food_price_index', 'trust_food_price_index'])

# Convert 'Date' column to datetime format
df_nofpi_range['Date'] = pd.to_datetime(df_nofpi_range['Date'])

#dropping columns related to inflation
df_noinf_range = df_nofpi_range.drop(columns=['inflation_beans','inflation_cabbage', 'inflation_carrots', 'inflation_eggs', 'inflation_garlic', 'inflation_meat_beef_chops', 'inflation_meat_chicken_whole', 'inflation_meat_pork', 'inflation_onions', 'inflation_potatoes', 'inflation_rice', 'inflation_tomatoes'])

#dropping columns related to trust scores
df_cleaned_range = df_noinf_range.drop(columns=['trust_beans','trust_cabbage', 'trust_carrots', 'trust_eggs', 'trust_garlic', 'trust_meat_beef_chops', 'trust_meat_chicken_whole', 'trust_meat_pork', 'trust_onions', 'trust_potatoes', 'trust_rice', 'trust_tomatoes'])

#dropping uneeded columns
df_unneeded_range = df_cleaned_range.drop(columns=['country', 'City', 'lat', 'lon', 'Province', 'Date', 'month'])

# Reshaping from wide to long format (including year and month as part of the identifier)
df_range = df_unneeded_range.melt(id_vars=['Region', 'year'], var_name='Food_Items', value_name='Price')

df_range_filtered = df_range[df_range['Food_Items'].str.startswith('c_')]

df_range_filtered.loc[:, 'Food_Items'] = df_range_filtered['Food_Items'].str.replace('c_', '', regex=True)

print(df_range_filtered)


                                   Region  year Food_Items  Price
354795   Cordillera Administrative region  2007      beans  84.71
354796   Cordillera Administrative region  2007      beans  84.03
354797   Cordillera Administrative region  2007      beans  83.63
354798   Cordillera Administrative region  2007      beans  83.91
354799   Cordillera Administrative region  2007      beans  83.76
...                                   ...   ...        ...    ...
1419175                    Market Average  2024   tomatoes  83.80
1419176                    Market Average  2024   tomatoes  83.34
1419177                    Market Average  2024   tomatoes  87.72
1419178                    Market Average  2024   tomatoes  79.34
1419179                    Market Average  2025   tomatoes  78.71

[283836 rows x 4 columns]


In [34]:
# Dash app setup
apph = dash.Dash(__name__)

apph.layout = html.Div([
    html.H3("Highest Prices Per Year by Region"),
    
    # Dropdown for selecting food items
    dcc.Dropdown(
        id='food-item-dropdown',
        options=[{'label': item, 'value': item} for item in df_range_filtered['Food_Items'].unique()],
        value=df_range_filtered['Food_Items'].unique()[0],
        placeholder="Select a food item"
    ),
    
    # Graph for displaying highest prices
    dcc.Graph(id='highest-price-graph')
])

# Callback to update the graph based on selected food item
@apph.callback(
    Output('highest-price-graph', 'figure'),
    [Input('food-item-dropdown', 'value')]
)
def update_graph(selected_food_item):
    # Filter data for the selected food item
    filtered_dfr = df_range_filtered[df_range_filtered['Food_Items'] == selected_food_item]
    
    # Find the highest price per year and the corresponding region
    highest_prices = (
        filtered_dfr.loc[filtered_dfr.groupby('year')['Price'].idxmax()]
        .reset_index(drop=True)
    )
    
    # Create the bar chart
    fig = px.bar(
        highest_prices,
        x='year',
        y='Price',
        color='Region',  # Highlight the region in the bar color
        title=f'Highest Prices Per Year for {selected_food_item}',
        labels={'Price': 'Price', 'year': 'Year', 'Region': 'Region'}
    )
    
    return fig

# Run the app
if __name__ == '__main__':
    apph.run_server(debug=True, port=8051, mode='inline', name="apph")

In [35]:
# Dash app setup
appl = dash.Dash(__name__)

appl.layout = html.Div([
    html.H3("lowest Prices Per Year by Region"),
    
    # Dropdown for selecting food items
    dcc.Dropdown(
        id='food-item-dropdown',
        options=[{'label': item, 'value': item} for item in df_range_filtered['Food_Items'].unique()],
        value=df_range_filtered['Food_Items'].unique()[0],
        placeholder="Select a food item"
    ),
    
    # Graph for displaying lowest prices
    dcc.Graph(id='lowest-price-graph')
])

# Callback to update the graph based on selected food item
@appl.callback(
    Output('lowest-price-graph', 'figure'),
    [Input('food-item-dropdown', 'value')]
)
def update_graph(selected_food_item):
    # Filter data for the selected food item
    filtered_dfr = df_range_filtered[df_range_filtered['Food_Items'] == selected_food_item]
    
    # Find the lowest price per year and the corresponding region
    lowest_prices = (
        filtered_dfr.loc[filtered_dfr.groupby('year')['Price'].idxmin()]
        .reset_index(drop=True)
    )
    
    # Create the bar chart
    fig = px.bar(
        lowest_prices,
        x='year',
        y='Price',
        color='Region',  # Highlight the region in the bar color
        title=f'Lowest Prices Per Year for {selected_food_item}',
        labels={'Price': 'Price', 'year': 'Year', 'Region': 'Region'}
    )
    
    return fig

# Run the app, changing port to make it have different URL and not interfere with other app
if __name__ == '__main__':
    appl.run_server(debug=True, port=8052, mode='inline', name="appl")

In [36]:
# Dash App Setup
apprange = dash.Dash(__name__)

apprange.layout = html.Div([
    html.H3("Price Range of Food Items per Year Across Regions"),
    
    # Dropdown for selecting food items
    dcc.Dropdown(
        id='food-item-dropdown',
        options=[{'label': item, 'value': item} for item in df_range_filtered['Food_Items'].unique()],
        value=df_range_filtered['Food_Items'].unique()[0],
        placeholder="Select a food item"
    ),
    
    # Graph for displaying the price range (min and max) per year
    dcc.Graph(id='price-range-graph')
])

# Callback to update the graph based on selected food item
@apprange.callback(
    Output('price-range-graph', 'figure'),
    [Input('food-item-dropdown', 'value')]
)
def update_graph(selected_food_item):
    # Filter data for the selected food item
    filtered_df = df_range_filtered[df_range_filtered['Food_Items'] == selected_food_item]
    
    # Get the minimum and maximum price per year
    price_range = (
        filtered_df.groupby('year')['Price']
        .agg(['min', 'max'])
        .reset_index()
    )
    
    # Create the graph (min and max prices as lines)
    fig = px.line(
        price_range, 
        x='year', 
        y=['min', 'max'], 
        title=f'Price Range (Min & Max) Per Year for {selected_food_item} Across Regions',
        labels={'year': 'Year', 'value': 'Price'},
        line_shape='linear'
    )
    
    return fig

# Run the app
apprange.run_server(debug=True, port=8053, mode='inline', name="apprange")

Key Observations:
1. Overall Trend - Both the minimum and maximum prices show a general upward trend over the years, indicating an overall increase in the price of every food items. Some even recorded highest price increase on 2020, likely due to the COVID-19 pandemic and its impact on supply chains.
2. Price Range - The gap between the minimum and maximum prices for most items widens significantly in 2020 and 2021. This suggests that there were greater variations in prices across regions during these years.

Posssible causes:
1. COVID-19 Pandemic - The pandemic disrupted supply chains and increased demand for certain food items, which could have led to price volatility.
2. Regional Differences - Differences in the impact of the pandemic on regional economies and agricultural production could have contributed to price variations.

To truly know if the regions really have price difference over the years, hypothesis testing must be done. Please refer to <ins>..\Data\Processed\hypothesis_testing.ipynb</ins> to know why Kruskall-Wallis was used here.

In [37]:
#open the csv file that was already analyzed with kruskall wallis test.
hypothesis_results = "../Data/Processed/hypothesis_testing_result.csv"
hyp_df = pd.read_csv(hypothesis_results)

In [38]:
# View the first few rows of the dataset
print(hyp_df.head())

   Year  Food_Item  H-statistic        p-value           Result
0  2007      beans          NaN            NaN  Not Significant
1  2007    c_beans  1077.368029  2.491117e-218      Significant
2  2007  c_cabbage  1105.762042  2.066749e-224      Significant
3  2007  c_carrots  1093.290901  9.693593e-222      Significant
4  2007     c_eggs   905.101619  1.726764e-181      Significant


In [39]:
# Filter the dataframe to keep only rows where 'food_item' starts with 'c_'
hyp_df_filtered = hyp_df[hyp_df['Food_Item'].str.startswith('c_')]

print(hyp_df_filtered)

      Year    Food_Item  H-statistic        p-value       Result
1     2007      c_beans  1077.368029  2.491117e-218  Significant
2     2007    c_cabbage  1105.762042  2.066749e-224  Significant
3     2007    c_carrots  1093.290901  9.693593e-222  Significant
4     2007       c_eggs   905.101619  1.726764e-181  Significant
5     2007     c_garlic   393.719714   3.807277e-73  Significant
...    ...          ...          ...            ...          ...
1088  2025  c_meat_pork    44.757215   2.646148e-04  Significant
1089  2025     c_onions   101.512409   4.662566e-14  Significant
1090  2025   c_potatoes    90.632301   4.683642e-12  Significant
1091  2025       c_rice    67.209085   6.484158e-08  Significant
1092  2025   c_tomatoes    70.027332   2.133517e-08  Significant

[228 rows x 5 columns]


In [40]:
# removing prefix _c in the food items
hyp_df_filtered.loc[:, 'Food_Item'] = hyp_df_filtered['Food_Item'].str.replace('c_', '', regex=True)
print(hyp_df_filtered)

      Year  Food_Item  H-statistic        p-value       Result
1     2007      beans  1077.368029  2.491117e-218  Significant
2     2007    cabbage  1105.762042  2.066749e-224  Significant
3     2007    carrots  1093.290901  9.693593e-222  Significant
4     2007       eggs   905.101619  1.726764e-181  Significant
5     2007     garlic   393.719714   3.807277e-73  Significant
...    ...        ...          ...            ...          ...
1088  2025  meat_pork    44.757215   2.646148e-04  Significant
1089  2025     onions   101.512409   4.662566e-14  Significant
1090  2025   potatoes    90.632301   4.683642e-12  Significant
1091  2025       rice    67.209085   6.484158e-08  Significant
1092  2025   tomatoes    70.027332   2.133517e-08  Significant

[228 rows x 5 columns]


In [41]:
year_dropdown = widgets.Dropdown(
    options=hyp_df_filtered["Year"].unique(),
    value=hyp_df_filtered["Year"].min(),
    description="Select Year:",
    style={'description_width': 'initial'}
)

# Output widget to display the table
output = widgets.Output()

def update_table(change):
    with output:
        output.clear_output()
        display(hyp_df_filtered[hyp_df_filtered["Year"] == year_dropdown.value])

# Trigger update on change
year_dropdown.observe(update_table, names="value")

# Display widgets
display(year_dropdown, output)

# Initial Table Display
update_table(None)

Dropdown(description='Select Year:', options=(np.int64(2007), np.int64(2008), np.int64(2009), np.int64(2010), …

Output()

In [42]:
# or you can just count the number of unique Results
hyp_df_filtered["Result"].value_counts()

Result
Significant    228
Name: count, dtype: int64

This means all the items for all the regions over the year have different prices. But which regions have different prices?

In [65]:
# Define the available items and years for the dropdowns
items = df_range_filtered['Food_Items'].unique().tolist()
years = df_range_filtered['year'].unique().tolist()

# Step 1: Create the function that will update the graph based on selected item and year
def update_heatmap(selected_item, selected_year):
    # Filter for the selected item and year
    filter_df = df_range_filtered[(df_range_filtered['Food_Items'] == selected_item) & (df_range_filtered['year'] == selected_year)]
    
    # Perform Dunn's Test to check which regions differ
    regions = filter_df['Region'].unique()
    dunn_result = sp.posthoc_dunn(filter_df, val_col='Price', group_col='Region')

    # Prepare heatmap data (p-values matrix)
    p_values_matrix = np.zeros((len(regions), len(regions)))

    for i, region1 in enumerate(regions):
        for j, region2 in enumerate(regions):
            if region1 != region2:
                p_value = dunn_result.loc[region1, region2]
                p_values_matrix[i, j] = p_value
            else:
                p_values_matrix[i, j] = np.nan  # No need for comparison within the same region

    # Create a DataFrame for the heatmap using the regions as both rows and columns
    heatmap_df = pd.DataFrame(p_values_matrix, columns=regions, index=regions)

    # Replace NaN values with a large value (optional, to handle missing pairwise comparisons)
    heatmap_df = heatmap_df.fillna(1.0)  # Optionally replace NaNs with 1 (indicating no comparison)

    # Custom colorscale: green for 1 and red for 0
    custom_colorscale = [
        [0, "darkred"], # 0 -> red
        [0.166, "red"],
        [0.332, "lightcoral"],
        [0.5, "yellow"],
        [0.66, "palegreen"],
        [0.832, "green"],
        [1, "darkgreen"]   # 1 -> green  
    ]

    # Create the heatmap figure with custom colorscale
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_df.values,
        x=heatmap_df.columns,
        y=heatmap_df.index,
        colorscale=custom_colorscale,
        colorbar=dict(title='p-value'),
        zmin=0, zmax=1  # Normalize the range from 0 to 1
    ))

    # Update layout to improve axis labels and graph size
    fig.update_layout(
        title=f"Dunn's Test p-values for {selected_item} in {selected_year} by Region",
        xaxis_title='Region',
        yaxis_title='Region',
        autosize=False,  # Disable autosize
        width=800,  # Set width of the figure
        height=800,  # Set height of the figure
        margin=dict(l=100, r=100, t=100, b=100),  # Increase margins for better visibility
        xaxis=dict(tickmode='array', tickvals=heatmap_df.columns, ticktext=heatmap_df.columns, tickangle=45),  # Rotate x-axis labels for better readability
        yaxis=dict(tickmode='array', tickvals=heatmap_df.index, ticktext=heatmap_df.index, tickangle=0)  # Rotate y-axis labels for better readability
    )

    return fig

# Step 2: Create the Dash app
appDunns = dash.Dash(__name__)

# Define the layout with dropdowns and the graph
appDunns.layout = html.Div([
    html.H3("Price Comparison Heatmap by Region"),
    
    # Dropdown for selecting item
    html.Label('Select Item:'),
    dcc.Dropdown(
        id='item-dropdown',
        options=[{'label': item, 'value': item} for item in items],
        value=items[0],  # default value
        style={'width': '50%'}
    ),
    
    # Dropdown for selecting year
    html.Label('Select Year:'),
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': year, 'value': year} for year in years],
        value=years[0],  # default value
        style={'width': '50%'}
    ),
    
    # Graph for showing the heatmap
    dcc.Graph(id='heatmap-graph')
])

# Step 3: Create callback to update the heatmap based on dropdown selection
@appDunns.callback(
    dash.dependencies.Output('heatmap-graph', 'figure'),
    [dash.dependencies.Input('item-dropdown', 'value'),
     dash.dependencies.Input('year-dropdown', 'value')]
)
def update_graph(selected_item, selected_year):
    return update_heatmap(selected_item, selected_year)

# Run the app
if __name__ == '__main__':
    appDunns.run_server(debug=True, port=8054, mode='inline', name="appDunns")

This heatmap visualizes the results of a Dunn's post-hoc test for significant differences in prices of different food items between regions across the years.<br>
<space><br>
The color scale indicates the following:
*   Red (Close to 0): Indicates a statistically significant difference in bean prices between two regions.
*   Light Green to Yellow (Around 0.2 to 0.8): Suggests a possible difference, but not statistically significant based on the chosen significance level (typically 0.05)
*   Dark Green (Close to 0): Indicates no statistically significant difference in bean prices between the two regions.

#### Key Observations
1. Significant Differences Exist: The presence of so many red cells indicates that there are statistically significant differences in food prices in almost all regions at any given years.
2. No Significant Differences: patches of green indicate pairs of regions where the Dunn's test did not find a statistically significant difference in food prices, but there are only few of them.
3. Market Average Comparison: The "Market Average" row/column allows you to see how each region's prices compare to the overall average. Dark red cells in this row/column would indicate regions with prices significantly different from the market average.